# Check implementation of Hamberg et al model

In [1]:
import os

import chi
import chi.plots
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go

from model import define_hamberg_model, define_hamberg_population_model

## Define Hamberg model

In [2]:
mechanistic_model, parameters_df = define_hamberg_model()

## Define covariate model (ignoring other sources of IIV)

In [3]:
def get_typical_parameters(age, cyp2c9, vkorc1):
    """
    Returns typical model parameters for patient with input characteristics
    according to Hamberg et al.

    :param age: The age of the patient.
    :type age: int
    :param cyp2c9: One of 5 variants of the CYP2C9 gene:
        ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    :type cyp2c9: str
    :param vkorc1: One of 5 variants of the VKORC1 gene:
        ['GG', 'GA', 'AA']
    :type vkorc1: str
    """
    cyp2c9_variants = ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    if cyp2c9 not in cyp2c9_variants:
        raise ValueError(
            'The provided CYP2C9 genotype is not among the allowed variants.')
    vkorc1_variants = ['GG', 'GA', 'AA']
    if vkorc1 not in vkorc1_variants:
        raise ValueError(
            'The provided VKORC1 genotype is not among the allowed variants.')

    # Define covariate independent parameters
    effective_volume_central = 14.3         # in L
    transition_rate_chain_1 = 3 / 28.6      # in 1/h
    transition_rate_chain_2 = 3 / 118.3     # in 1/h

    # Compute clearance
    clearance_per_allele_1 = 0.174          # in L/h
    clearance_per_allele_2 = 0.0879         # in L/h
    clearance_per_allele_3 = 0.0422         # in L/h

    if cyp2c9 == '*1/*1':
        clearance = 2 * clearance_per_allele_1
    if cyp2c9 == '*1/*2':
        clearance = clearance_per_allele_1 + clearance_per_allele_2
    if cyp2c9 == '*1/*3':
        clearance = clearance_per_allele_1 + clearance_per_allele_3
    if cyp2c9 == '*2/*2':
        clearance = 2 * clearance_per_allele_2
    if cyp2c9 == '*2/*3':
        clearance = clearance_per_allele_2 + clearance_per_allele_3
    if cyp2c9 == '*3/*3':
        clearance = 2 * clearance_per_allele_3

    change_per_year = -0.00571              # rel. change centered around 71 y
    clearance *= (1 + change_per_year * (age - 71))

    # Compute EC50 of warfarin
    ec50_per_allele_A = 0.96                # in mg/L
    ec50_per_allele_G = 2.05                # in mg/L

    if vkorc1 == 'GG':
        half_maximal_effect_conc = 2 * ec50_per_allele_G
    if vkorc1 == 'GA':
        half_maximal_effect_conc = ec50_per_allele_G + ec50_per_allele_A
    if vkorc1 == 'AA':
        half_maximal_effect_conc = 2 * ec50_per_allele_A

    parameters = [
        effective_volume_central,
        clearance,
        half_maximal_effect_conc,
        transition_rate_chain_1,
        transition_rate_chain_2
    ]

    return parameters


## Reproduce Figure 3 in Hamberg et al

In [4]:
# Define simulation conditions
thirty_days = 30 * 24
times = np.linspace(0, thirty_days, num=1000)

# I: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GG VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GG'
doses = [7.6, 3.8, 1.8]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation1 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    # (Model only captures S-warfarin, so need to divide dose by 2)
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose/2, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation1.append(mechanistic_model.simulate(params, times))

# II: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GA'
doses = [5.6, 2.8, 1.4]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation2 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose/2, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation2.append(mechanistic_model.simulate(params, times))

# III: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'AA'
doses = [3.6, 1.8, 0.9]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation3 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose/2, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation3.append(mechanistic_model.simulate(params, times))

# Figure 1: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
times = times / 24
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][0],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][0],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][0],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 2: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][1],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][1],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][1],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Warfarin concentration in mg/L',
)
fig.show()

# Figure 3: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][0],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][0],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][0],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 4: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][1],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][1],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][1],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 5: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][0],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][0],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][0],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 6: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][1],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][1],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][1],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()


## Check that inference can recover the model parameters

### Generate demographic data

In [5]:
# Define population model
population_model = define_hamberg_population_model()

# Define covariates
# Frequency of alleles matches dataset in publication
n_ids = 1000
n_cov = 3
covariates = np.zeros(shape=(n_ids, n_cov))
n_cyp2p9_33 = int(np.ceil(0.006 * n_ids))
covariates[:n_cyp2p9_33, 0] += 1
n_cyp2p9_23 = int(np.ceil(0.012 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23, 0] += 1
n_cyp2p9_22 = int(np.ceil(0.014 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22, 0] += 1
n_cyp2p9_13 = int(np.ceil(0.123 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22+n_cyp2p9_13, 0] += 1
n_cyp2p9_12 = int(np.ceil(0.184 * n_ids))
covariates[
    :n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22+n_cyp2p9_13+n_cyp2p9_12, 0] += 1

typical_age = 68
covariates[:, 1] = np.random.lognormal(
    mean=np.log(typical_age), sigma=0.1, size=n_ids)

n_vkorc1_AA = int(np.ceil(0.15 * n_ids))
covariates[:n_vkorc1_AA, 2] += 1
n_vkorc1_GA = int(np.ceil(0.485 * n_ids))
covariates[:n_vkorc1_AA+n_vkorc1_GA, 2] += 1

# Sample individual-level parameters
seed = 1
data_generating_parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in population_model.get_parameter_names()
])
psi = population_model.sample(
    parameters=data_generating_parameters, covariates=covariates,
    n_samples=n_ids, seed=seed)

### Visualise demographic data

In [6]:
# Plot volume of distribution
fig = ff.create_distplot(
    [psi[:, 0]], ['Patients'], bin_size=.2, show_hist=True,
    show_curve=False)
fig.update_layout(xaxis_title='Volume of distribution in L')
fig.show()

# Plot clearance
n_1 = n_cyp2p9_33
n_2 = n_cyp2p9_33 + n_cyp2p9_23
n_3 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22
n_4 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22 + n_cyp2p9_13
n_5 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22 + n_cyp2p9_13 + n_cyp2p9_12
fig = ff.create_distplot(
    [
        psi[:n_1, 1],
        psi[n_1:n_2, 1],
        psi[n_2:n_3, 1],
        psi[n_3:n_4, 1],
        psi[n_4:n_5, 1],
        psi[n_5:, 1]
    ],
    ['*3/*3', '*2/*3', '*2/*2', '*1/*3', '*1/*2', '*1/*1'], bin_size=.005,
    show_hist=True, show_curve=False)
fig.update_layout(xaxis_title='Clearance in L/h')
fig.show()

# Plot age distribution
fig = ff.create_distplot(
    [covariates[:, 1]], ['Patients'], show_hist=True,
    show_curve=False)
fig.update_layout(xaxis_title='Age in years')
fig.show()

# Plot EC50
fig = ff.create_distplot(
    [
        psi[:n_vkorc1_AA, 2],
        psi[n_vkorc1_AA:n_vkorc1_GA, 2],
        psi[n_vkorc1_AA+n_vkorc1_GA:, 2]
    ],
    ['AA', 'GA', 'GG'], bin_size=.05, show_hist=True, show_curve=False)
fig.update_layout(xaxis_title='EC50 in mg/L')
fig.show()

### Generate INR data

In [7]:
# Define measurement model for an individual
error_models = [
    chi.LogNormalErrorModel(),  # Warfarin concentration
    chi.LogNormalErrorModel()]  # INR
predictive_model = chi.PredictiveModel(mechanistic_model, error_models)

# Select patients
seed = 12
np.random.seed(seed)
n_measured_individuals = 100
indices = np.random.choice(
    np.arange(n_ids), replace=False, size=n_measured_individuals)
patients = np.copy(psi[indices])
patient_cov = np.copy(covariates[indices])

# Shuffle covariates
indices = np.random.choice(
    np.arange(n_measured_individuals), replace=False,
    size=n_measured_individuals)
patients[:, 1] = patients[indices, 1]
patient_cov[:, 0] = patient_cov[indices, 0]

# Sample INR measurements
seed = 42
times = np.linspace(0, 15, num=100) * 24
predictive_model.set_dosing_regimen(dose=4, start=0, period=24)
measurements = pd.DataFrame(
    columns=['ID', 'Time', 'Observable', 'Value', 'Dose', 'Duration'])
for idp, patient in enumerate(patients):
    data = predictive_model.sample(
        parameters=patient, times=times, include_regimen=True, seed=seed+idp)
    data = pd.concat([data, pd.DataFrame({
        'ID': idp + 1,
        'Time': np.nan,
        'Observable': ['CYP2C9', 'Age', 'VKORC1'],
        'Value': patient_cov[idp],
        })])
    data['ID'] = idp + 1
    measurements = pd.concat([measurements, data])

# Export measurements
directory = os.path.dirname(os.getcwd())
measurements.to_csv(directory + '/data/synthetic_hamberg_model_data.csv')

### Visualise INR measurements

In [8]:
# Visualise measurements
temp = measurements.copy()
temp['Time'] /= 24

# Get only 10 individuals as example
ids = temp.ID.dropna().unique()
mask = temp.ID == ids[0]
for _id in ids[np.random.randint(0, len(ids), size=10)]:
    mask = mask | (temp.ID == _id)
temp = temp[mask]

fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(temp, observable='central.s_warfarin_concentration')
fig.set_axis_labels(xlabel='Time in days', ylabel='Warfarin conc. in mg/L')
fig.show()

fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(temp, observable='myokit.inr')
fig.set_axis_labels(xlabel='Time in days', ylabel='INR')
fig.show()

## Visualise inference results

In [6]:
import pints

In [28]:
p = pints.ComposedLogPrior(
        pints.GaussianLogPrior(2.7, 0.5),    # Mean log volume
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log volume
        pints.GaussianLogPrior(-1, 0.5),     # Mean log clearance
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log clearance
        pints.LogNormalLogPrior(-1.6, 0.8),  # Rel. shift clearance CYP29P *2
        pints.LogNormalLogPrior(-1.6, 0.8),  # Rel. shift clearance CYP29P *3
        pints.GaussianLogPrior(0, 0.01),     # Rel. shift clearance Age
        pints.GaussianLogPrior(1.41, 0.5),   # Mean log EC50
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log EC50
        pints.LogNormalLogPrior(-1.6, 0.8),  # Rel. shift EC50 VKORC1 A
        pints.LogNormalLogPrior(-2.3, 0.8),  # Pooled rate chain 1
        pints.LogNormalLogPrior(-3.7, 1.5),  # Pooled rate chain 2
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log drug conc.
        pints.LogNormalLogPrior(0.1, 0.3)    # Sigma log INR
    )

test = p.sample()[0]
population_model.sample(test, covariates=covariates[0])

Clearance:  0.91144397123858
0.949049921573577
0.82288794247716
0.860493892812157
0.8980998431471542
[[0.99202853]]
EC50:  0.9260125463355164
0.8520250926710329


array([[2.67396451e+01, 1.60084077e-01, 1.18212792e+01, 7.37758226e-02,
        2.02636223e-02, 1.45360756e+00, 1.18714239e+00]])

In [23]:
population_model.sample(test, covariates=covariates[0])

Clearance:  0.935149864741885
0.799984587214025
0.8702997294837699
0.73513445195591
0.59996917442805
[[0.96295153]]
EC50:  0.9354561410972114
0.8709122821944228


array([[1.70081865e+01, 8.72193920e-01, 1.49139024e+01, 2.67877111e-01,
        8.61405253e-03, 7.47444676e-01, 2.02087862e+00]])

In [4]:
import chi
import numpy as np
from model import HambergClearanceCovariateModel

model = chi.CovariatePopulationModel(
        chi.LogNormalModel(), HambergClearanceCovariateModel())

In [12]:
model.n_parameters()

5

In [8]:
from model import HambergClearanceCovariateModel

model = chi.CovariatePopulationModel(
        chi.LogNormalModel(), HambergClearanceCovariateModel())
epsilon = 0.00001
n_parameters = model.n_parameters()
parameters = np.full(shape=n_parameters, fill_value=0.1)
observations = np.random.lognormal(0.1, 0.1, size=6)
covariates = np.full(shape=(6, 2), fill_value=72)
covariates[:, 0] = np.arange(6)
ref_sens = []
for index in range(n_parameters):
    # Construct parameter grid
    low = parameters.copy()
    low[index] -= epsilon
    high = parameters.copy()
    high[index] += epsilon

    # Compute reference using numpy.gradient
    sens = np.gradient(
        [
            model.compute_log_likelihood(low, observations, covariates),
            model.compute_log_likelihood(parameters, observations, covariates),
            model.compute_log_likelihood(high, observations, covariates)],
        (epsilon))
    ref_sens.append(sens[1])

# Compute sensitivities with hierarchical model
_, dobs, sens = model.compute_sensitivities(parameters, observations, covariates)

In [9]:
ref_sens

[103.76983498610048,
 168.69950078621798,
 -48.644672792619296,
 -45.614595261067585,
 -114.112366374286]

In [10]:
sens

array([ 103.76983499,  168.69949641,  -48.64467278,  -45.61459525,
       -114.11613745])

In [29]:
epsilon = 0.00001
n_parameters = model.n_parameters()
parameters = np.full(shape=n_parameters, fill_value=0.1)
observations = np.random.lognormal(0.1, 0.1, size=3)
covariates = np.arange(3)[:, np.newaxis]
ref_sens = []
for index in range(3):
    # Construct parameter grid
    low = observations.copy()
    low[index] -= epsilon
    high = observations.copy()
    high[index] += epsilon

    # Compute reference using numpy.gradient
    sens = np.gradient(
        [
            model.compute_log_likelihood(parameters, low, covariates),
            model.compute_log_likelihood(parameters, observations, covariates),
            model.compute_log_likelihood(parameters, high, covariates)],
        (epsilon))
    ref_sens.append(sens[1])

# Compute sensitivities with hierarchical model
_, dobs, sens = model.compute_sensitivities(parameters, observations, covariates)

In [30]:
ref_sens

[-3.862263783371844, -18.746957540616283, -5.13508099538873]

In [31]:
dobs

array([[ -3.86226379],
       [-18.74695754],
       [ -5.135081  ]])